# 新组件开发教程
TrustFlow中组件定义和实现都在[Teeapps](https://github.com/asterinas/trustflow-teeapps)中。
我们的组件通过secretflow的[component spec](https://github.com/secretflow/spec)来统一定义。这是隐语开放标准中用于定义组件的标准。用这套标准我们可以定义组件的名称、版本，参数的类型、取值范围、说明，定义输入输出的格式。建议您先阅读这套组件定义标准，便于理解接下来的开发流程。

下面我们以新增LightGBM训练算法为例来说明新增组件的开发流程。
准备好代码：
```bash
git clone https://github.com/asterinas/trustflow-teeapps.git
```

## 定义组件

### 1. 新建文件
在`teeapps/component`目录下找到合适的组件分类，您也可以新建新的组件分类。
LightGBM训练算法属于机器学习领域，因此在`ml/train`分类下新建`lgbm_component.h`和`lgbm_component.cc`文件。

### 2. 声明组件
LightGBM训练组件头文件`lgbm_component.h`示例如下：
```c++
#pragma once

#include "../../component.h"

namespace teeapps {
namespace component {

class LgbmTrainComponent : public Component {
 private:
  void Init();

  explicit LgbmTrainComponent(
      const std::string& name = "lgbm_train",
      const std::string& domain = "ml.train",
      const std::string& version = "0.0.1",
      const std::string& desc =
          "LightGBM train component for individual dataset.")
      : Component(name, domain, version, desc) {
    Init();
  }
  ~LgbmTrainComponent() {}
  LgbmTrainComponent(const LgbmTrainComponent&) = delete;
  const LgbmTrainComponent& operator=(const LgbmTrainComponent&) = delete;

 public:
  static LgbmTrainComponent& GetInstance() {
    static LgbmTrainComponent instance;
    return instance;
  }
};

}  // namespace component
}  // namespace teeapps
```

这段代码声明了新组件的一些基本信息:

- 组件名称："lgbm_train"
- 组件所属领域："ml.train"
- 组件版本号："0.0.1"
- 组件描述："LightGBM train component for individual dataset."

### 3. 定义组件参数
我们在`lgbm_component.cc`中来定义组件的详细参数，包含每个参数的名字、类型、取值范围、默认值、是否是可选参数等。
添加参数需要用到的`AddAttr`函数的具体定义如下：

```c++
template <typename T>
void AddAttr(
      const std::string& name, const std::string& desc, bool is_list,
      bool is_optional,
      const std::optional<std::vector<T>>& default_values = std::nullopt,
      const std::optional<std::vector<T>>& allowed_values = std::nullopt,
      const std::optional<T>& lower_bound = std::nullopt,
      const std::optional<T>& upper_bound = std::nullopt,
      const std::optional<bool>& lower_bound_inclusive = std::nullopt,
      const std::optional<bool>& upper_bound_inclusive = std::nullopt,
      const std::optional<int>& list_min_length_inclusive = std::nullopt,
      const std::optional<int>& list_max_length_inclusive = std::nullopt);
```
- name：参数名称。
- desc：参数的详细描述。
- is_list：参数是否是一个列表。如果为fasle，则代表了我们允许用户输入一个T类型的值；如果是true，则代表了允许用户输入一个T类型的列表。
- is_optional：参数是否是optional的。如果为true，则代表了用户可以不填，此时会使用该参数的默认值；如果为false，则代表用户必须传递该值。
- default_values：参数的默认值，std::nullopt表示不设置默认值。当is_optional为true时必须定义该默认值。
- allowed_values：参数的允许值，std::nullopt表示不设置允许值。如果设置了该值，那么用户就必须在给出的allowed_values中选择输入。
- lower_bound：参数下限，std::nullopt表示不设置下限。
- lower_bound_inclusive：下限是否是包含。如果为true，则代表了lower_bound也是一个合法的输入。
- upper_bound：参数上限，std::nullopt表示不设置上限。
- upper_bound_inclusive：上限是否包含。如果为true，则代表了upper_bound也是一个合法的输入。
- list_min_length_inclusive：列表类型参数的最小长度，std::nullopt表示不设置列表最小长度。该值仅在is_list为true的时候可选设置。
- list_max_length_inclusive：列表类型参数的最大长度，std::nullopt表示不设置列表最大长度。该值仅在is_list为true的时候可选设置。

我们先添加一个训练轮数的参数:
```c++
#include "lgbm_component.h"

namespace teeapps {
namespace component {

void LgbmTrainComponent::Init() {
  AddAttr<int64_t>("n_estimators", "Number of boosted trees to fit.", false,
                   true, std::vector<int64_t>{10}, std::nullopt, 1, 1024, true,
                   true);
}

}  // namespace component
}  // namespace teeapps
```
这段代码使用`AddAttr`函数为`lgbm_train`这个组件添加了一个参数：

- 参数名称："n_estimators"
- 参数详细描述："Number of boosted trees to fit."
- 非列表类型参数，代表输入的是一个值而非一个列表
- 可选参数，代表用户如果不填该参数，就会使用默认值
- 参数的默认值为10
- 不设定特定的某几个可选值，但是设定了取值范围为[1, 1024]，包含下限1和上限1024。

接下来，您可以在`lgbm_component.cc`中继续添加您需要的参数，下面为一个示例，您可以根据需要进行删改：
```c++
#include "lgbm_component.h"

namespace teeapps {
namespace component {

void LgbmTrainComponent::Init() {
  AddAttr<int64_t>("n_estimators", "Number of boosted trees to fit.", false,
                   true, std::vector<int64_t>{10}, std::nullopt, 1, 1024, true,
                   true);
  AddAttr<std::string>("objective", "Specify the learning objective.", false,
                       true, std::vector<std::string>{"binary"},
                       std::vector<std::string>{"regression", "binary"});
  AddAttr<std::string>("boosting_type", "Boosting type.", false, true,
                       std::vector<std::string>{"gbdt"},
                       std::vector<std::string>{"gbdt", "rf", "dart"});
  AddAttr<float>("learning_rate", "Learning rate.", false, true,
                 std::vector<float>{0.1}, std::nullopt, 0, 1, false, true);
  AddAttr<int64_t>("num_leaves", "Max number of leaves in one tree.", false,
                   true, std::vector<int64_t>{31}, std::nullopt, 2, 1024, true,
                   true);
}

}  // namespace component
}  // namespace teeapps
```

### 4. 定义组件输入输出
我们继续在`lgbm_component.cc`中来定义组件的输入输出。
定义输入输出需要用到的`AddIo`函数定义如下：
```c++
void AddIo(const IoType io_type, const std::string& name,
             const std::string& desc, const std::vector<std::string>& types,
             const std::optional<std::vector<TableColParam>>& col_params =
                 std::nullopt);
```
- io_type：表示这是输入还是输出，可选IoType::INPUT和IoType::OUTPUT。
- name：io名称。
- desc：io的详细说明。
- types: io的类型，使用定义在teeapps/component/util.h的DistDataType中的字符串作为类型名称，如果需要添加新的类型，请在DistDataType中添加定义。
- col_params:  额外列参数。对于输入的数据表，如果需要额外指定一些列参数，可以在这一项进行设置。比如在psi组件中，可以用"key"指定哪一列用于求交；比如在woe组件中，可以用"feature_selects"指定对哪些列进行woe binning；再比如训练组件中可以用"label"指定哪一列作为训练标签。


定义输入输出：
```c++
#include "lgbm_component.h"

namespace teeapps {
namespace component {

void LgbmTrainComponent::Init() {
  //省略了前面定义的组件参数....

  AddIo(IoType::INPUT, "train_dataset", "Input table.",
        {DistDataType::INDIVIDUAL_TABLE},
        std::vector<TableColParam>{
            TableColParam("ids", "Id columns will not be trained."),
            TableColParam("label", "Label column.", 1, 1)});
  AddIo(IoType::OUTPUT, "output_model", "Output model.",
        {DistDataType::LGBM_MODEL});
}

}  // namespace component
}  // namespace teeapps
```

这段代码定义了`lgbm_train`组件的输入输出：

- 输入名称为"train_dataset"
- 输入的类型为DistDataType::INDIVIDUAL_TABLE，单边表，通常就是一个csv表格
- 输入的额外列参数"ids"：指明了哪些列作为id列，没有设定数量的上下限，也就是可以不指定id列，也可以指定多列都作为id列，被作为id列的那些列不会被用于训练。
- 输入的额外列参数"label"：指明了哪一列作为训练的标签列，设定数量的上下限均为1，也就是有且仅有一列标签用于训练（不同于数据表的schema中可以有多列label，训练时必须指明一列作为训练标签）。




## 编写组件执行逻辑

Teeapps框架会解析json化的[sf_node_eval_param](https://github.com/secretflow/spec/blob/main/secretflow/spec/v1/evaluation.proto)，检查参数范围，对默认值进行赋值等，然后生成一个执行时的配置文件。组件执行代码可以直接读取该配置文件中的参数值，配置文件格式示例如下：
```json
{
  "component_name": "lgbm_train",
  "n_estimators": 10,
  "objective": "binary",
  "boosting_type": "gbdt",
  "num_leaves": 15,
  "learning_rate": 0.1,
  "inputs": [
    {
      "data_path": "teeapps/biz/testdata/breast_cancer/breast_cancer.csv",
      "schema": {
        "ids": [
          "id"
        ],
        "features": [
          "mean radius",
          "mean texture",
          "mean perimeter",
          "mean area",
          "mean smoothness",
          "mean compactness",
          "mean concavity",
          "mean concave points",
          "mean symmetry",
          "mean fractal dimension"
        ],
        "labels": [
          "target"
        ],
        "id_types": [
          "int"
        ],
        "feature_types": [
          "float",
          "float",
          "float",
          "float",
          "float",
          "float",
          "float",
          "float",
          "float",
          "float"
        ],
        "label_types": [
          "bool"
        ]
      },
      "ids": ["id"],
      "label": ["target"]
    }
  ],
  "outputs": [
    {
      "data_path": "lgbm_bin_class.model"
    }
  ]
}
```

在`teeapps/biz`目录下新建`lgbm/lgbm.py`实现组件执行逻辑，它将按照上述json中的配置执行相应算法：
```python
import json
import logging
import sys

import joblib
import lightgbm as lgb
import pandas

from teeapps.biz.common import common

COMPONENT_NAME = "lgbm_train"

IDS = "ids"
LABEL = "label"

N_ESTIMATORS = "n_estimators"
OBJECTIVE = "objective"
BOOSTING_TYPE = "boosting_type"
LEARNING_RATE = "learning_rate"
NUM_LEAVES = "num_leaves"

REGRESSION = "regression"
BINARY = "binary"


def run_lgbm(task_config: dict):
    logging.info("Running lgbm training...")

    assert (
        task_config[common.COMPONENT_NAME] == COMPONENT_NAME
    ), f"Component name should be {COMPONENT_NAME}, but got {task_config[common.COMPONENT_NAME]}"

    inputs = task_config[common.INPUTS]
    outputs = task_config[common.OUTPUTS]

    assert len(inputs) == 1, f"{COMPONENT_NAME} should have only 1 input"
    assert len(outputs) == 1, f"{COMPONENT_NAME} should have only 1 output"

    # get train data
    logging.info("Loading training data...")
    df = common.gen_data_frame(inputs[0])

    # labels in schema can be multiple, but eval target label is unique(in params)
    ids = inputs[0][IDS]
    labels = inputs[0][LABEL]
    assert len(labels) == 1, f"{COMPONENT_NAME} should have only 1 labels column"

    features = inputs[0][common.SCHEMA][common.FEATURES]
    features = [feature for feature in features if feature not in ids + labels]

    X = df[features]
    Y = pandas.to_numeric(df[labels[0]], errors="coerce")

    param = dict()
    param_keys = [N_ESTIMATORS, OBJECTIVE, BOOSTING_TYPE, LEARNING_RATE, NUM_LEAVES]

    for key in param_keys:
        param[key] = task_config[key]

    if param[OBJECTIVE] == REGRESSION:
        model = lgb.LGBMRegressor(**param)
    elif param[OBJECTIVE] == BINARY:
        model = lgb.LGBMClassifier(**param)
    else:
        raise RuntimeError(f"unsupported objective function: {param[OBJECTIVE]}")

    # train model
    model.fit(X, Y)

    logging.info("Setting origin feature_name in model...")
    model.origin_feature_name_ = features

    # dump model
    logging.info("Dumping model...")
    model_data_path = outputs[0][common.DATA_PATH]
    joblib.dump(model, model_data_path)


def main():
    assert len(sys.argv) == 2, f"Wrong arguments number: {len(sys.argv)}"
    # load task_config json
    task_config_path = sys.argv[1]
    logging.info("Reading task config file...")
    with open(task_config_path, "r") as task_config_f:
        task_config = json.load(task_config_f)
        logging.debug(f"Configurations: {task_config}")
        run_lgbm(task_config)


"""
This app is expected to be launched by app framework via running a subprocess 
`python3 lgbm.py config`. Before launching the subprocess, the app framework will 
firstly generate a config file which is a json file containing all the required 
parameters and is serialized from the task.proto. Currently we do not handle any 
errors/exceptions in this file as the outer app framework will capture the stderr 
and stdout.
"""
if __name__ == "__main__":
    # TODO set log level
    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
    )
    main()
```

## 注册组件

### 1. 在`teeapps/component/component_list.h`中注册组件

在ComponentDomain中新增Domain名，没有新增Domain则不需要添加。

在ComponentName中新增组件名:

```c++
struct ComponentName {
  ...
  static constexpr char kLgbmTrainComp[] = "lgbm_train";
  ...
};
```

在ComponentPyFile中新增组件执行逻辑的python文件名:

```c++
struct ComponentPyFile {
  ...
  static constexpr char kLgbmPy[] = "lgbm.py";
  ...
};
```

在comp_py_map中新增组件名与组件执行python文件名的映射关系:

```c++
const std::unordered_map<std::string, std::string> comp_py_map = {
    ...
    {ComponentName::kLgbmTrainComp, ComponentPyFile::kLgbmPy},
    ...
};
```

在COMP_DEF_MAP中新增组件全名与组件定义的映射关系:

```c++
const std::map<std::string, secretflow::spec::v1::ComponentDef> COMP_DEF_MAP = {
    ...
    {GenCompFullName(ComponentDomain::kMlTrainDomain,
                     ComponentName::kLgbmTrainComp, kCompVersion),
     secretflow::spec::v1::ComponentDef(
         *teeapps::component::LgbmTrainComponent::GetInstance().Definition())},
    ...
    };
```

### 2. 增加翻译（可选）
在teeapps/component/all_translation_cn.json中增加组件名称和参数的翻译，例如：
```json
{
  ...
  "ml.train/lgbm_train:0.0.1": {
    "ml.train": "模型训练",
    "lgbm_train": "LightGBM训练",
    "LightGBM train component for individual dataset.": "为独立数据集提供LightGBM训练能力的组件",
    "0.0.1": "0.0.1",
    "n_estimators": "训练轮数",
    "Number of boosted trees to fit.": "训练轮数",
    "objective": "学习目标",
    "Specify the learning objective.": "指定学习目标（二分类或回归）",
    "boosting_type": "基学习类型",
    "Boosting type.": "基学习类型",
    "learning_rate": "学习率",
    "Learning rate.": "学习率",
    "num_leaves": "叶子数",
    "Max number of leaves in one tree.": "一棵树中的最大叶子数量",
    "train_dataset": "训练数据集",
    "Input table.": "输入的训练数据集",
    "ids": "id列",
    "Id columns will not be trained.": "指定的id列不会作为训练的特征",
    "label": "标签列",
    "Label column.": "标签列",
    "output_model": "输出模型",
    "Output model.": "输出模型"
  },
  ...
}
```

### 3. 生成新的组件列表(可选)
进入开发容器
```bash
bash env.sh
bash env.sh enter
```

编译component目录
```bash
bazel --output_base=target build //teeapps/component/...
```

生成组件列表和翻译列表
```bash
./bazel-bin/teeapps/component/main
```
您将在`teeapps/component/comp_list.json`中看到新的组件列表，它对应secretpad中的[trustflow组件定义](https://github.com/secretflow/secretpad/blob/main/config/components/trustflow.json)
在`teeapps/component/translation.json`中看到相关字段的翻译，它对应secretpad中的[trustflow组件翻译](https://github.com/secretflow/secretpad/blob/main/config/i18n/trustflow.json)。


## 构建Teeapps镜像
在主机上用`deployment`目录下的`build.sh`脚本来构建不同平台下的镜像。

对于sgx平台，在运行脚本前还需要在`deployment/occlum/python.yaml`中添加`lgbm.py`，如下：

```yaml
includes:
  - base.yaml
targets:
  - target: /bin
    createlinks:
      - src: /opt/python-occlum/bin/python3
        linkname: python3
  # python packages
  - target: /opt
    copy: 
      - dirs:
          - /home/teeapp/python-occlum
  - target: /
    copy:
      - from: /home/teeapp/occlum/teeapps/biz
        dirs:
          - secretflow
          - teeapps
        files: 
          - biclassification_eval.py
          - feature_filter.py
          - train_test_split.py
          - lr.py
          - predict.py
          - prediction_bias_eval.py
          - psi.py
          - pearsonr.py
          - vif.py
          - table_statistics.py
          - woe_binning.py
          - woe_substitution.py
          - xgb.py
          - lgbm.py
          - __init__.py
```

`build.sh`镜像构建脚本执行方式如下:

```bash
cd deployment

bash build.sh -p sim -v ${VERSION}

bash build.sh -p sgx -v ${VERSION}

bash build.sh -p tdx -v ${VERSION}

bash build.sh -p csv -v ${VERSION}
```